# 2. Analisi Esplorativa dei Dati (EDA)

**Obiettivo:** Analizzare il dataset pulito (`players_cleaned.parquet`) per scoprire i primi pattern, identificare trend e capire le relazioni tra le statistiche. Questa fase è fondamentale per farmi un'idea dei dati e guidare le fasi successive.

1.  **Caricamento Dati:** Leggo il DataFrame pulito che ho preparato prima.
2.  **Analisi dei Leader Statistici:** Identifico i migliori giocatori per punti, rimbalzi e assist nella stagione 2023. È un buon modo per verificare che i dati siano coerenti con la realtà.
3.  **Analisi di Distribuzione:** Guardo come sono distribuiti i punti per partita per capire la concentrazione del talento offensivo.
4.  **Analisi di Correlazione:** Creo una heatmap per vedere quali statistiche tendono a muoversi insieme.

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import desc

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.config.spark_config import SPARK_CONFIG
from src.utils.helpers import get_spark_session

project_root = os.path.abspath(os.path.join('..'))
processed_data_dir = os.path.join(project_root, "data", "processed")
cleaned_df_path = os.path.join(processed_data_dir, "players_cleaned.parquet")

spark = get_spark_session(
    app_name="NBA_EDA",
    driver_memory=SPARK_CONFIG["driver_memory"]
)

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

### Fase 1: Caricamento dei Dati Puliti

Inizio caricando il DataFrame `players_cleaned.parquet` e verifico che lo schema sia corretto.

In [ ]:
df = spark.read.parquet(cleaned_df_path)

print("Schema del DataFrame caricato:")
df.printSchema()

#### Interpretazione dell'output

Ottimo, lo schema è corretto. Le colonne numeriche sono `double`, quindi sono pronto per iniziare l'esplorazione.

### Fase 2: Analisi dei Leader Statistici (Stagione 2023)

Per assicurarmi che i dati siano sensati, filtro per la stagione 2023 e vedo chi sono i leader nelle statistiche principali. È una sorta di "controllo di sanità mentale".

In [ ]:
df_2023 = df.filter(df.season == 2023)

print("Top 10 Marcatori (Punti Totali) - Stagione 2023")
df_2023.orderBy(desc("pts")).select("player", "pts").show(10)

print("Top 10 Rimbalzisti (Rimbalzi Totali) - Stagione 2023")
df_2023.orderBy(desc("trb")).select("player", "trb").show(10)

print("Top 10 Assistmen (Assist Totali) - Stagione 2023")
df_2023.orderBy(desc("ast")).select("player", "ast").show(10)

#### Interpretazione dell'output

I nomi che vedo (Tatum, Sabonis, Young) sono quelli che mi aspettavo. Questo mi conferma che i dati sono accurati e posso fidarmi per le analisi successive.

### Fase 3: Distribuzione dei Punti per Partita (Stagione 2023)

Creo un istogramma per vedere come si distribuisce la media punti per partita. Questo mi aiuta a capire quanti giocatori sono d'élite, quanti sono nella media e quanti sono giocatori di ruolo.

In [ ]:
ppg_df = df_2023.withColumn("ppg", df_2023.pts / df_2023.g).select("ppg").toPandas()

plt.figure(figsize=(14, 7))
sns.histplot(ppg_df['ppg'], bins=30, kde=True)
plt.title('Distribuzione dei Punti per Partita (PPG) - Stagione 2023', fontsize=16)
plt.xlabel('Punti per Partita (PPG)', fontsize=12)
plt.ylabel('Numero di Giocatori', fontsize=12)
plt.show()

#### Interpretazione del grafico

Il grafico mostra una distribuzione asimmetrica a destra. La maggior parte dei giocatori si concentra nella fascia bassa (0-10 PPG), mentre i realizzatori d'élite sono una piccola minoranza. È una forma tipica delle distribuzioni di talento.

### Fase 4: Correlazione tra le Statistiche di Base

Uso una heatmap per vedere quali statistiche sono correlate tra loro. È un modo veloce per scoprire relazioni interessanti.

In [ ]:
stats_df = df_2023.select("pts", "trb", "ast", "stl", "blk").toPandas()

plt.figure(figsize=(10, 8))
correlation_matrix = stats_df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matrice di Correlazione tra Statistiche di Base - Stagione 2023', fontsize=16)
plt.show()

#### Interpretazione del grafico

La heatmap mi mostra che c'è una forte correlazione positiva tra punti e assist (0.69), tipica delle superstar. C'è anche una correlazione moderata tra rimbalzi e stoppate (0.50), tipica dei lunghi. Questo mi suggerisce già che esistono diversi "profili" di giocatori, che andrò a definire formalmente con il clustering.

### Conclusione e Prossimi Passi

L'analisi esplorativa mi ha dato delle buone intuizioni. Ho verificato la qualità dei dati e ho già intravisto delle relazioni interessanti.

Nel prossimo notebook, `03_advanced_metrics_calculation.ipynb`, passerò al calcolo di metriche più avanzate per descrivere meglio le performance.

In [ ]:
spark.stop()